# Preprocessing

This is the preprocessing notebook related to the following notebook:


[CNN+RNN]CNN pretraining w/ regression -TRAIN

Many thanks to [Y.Nakama](https://www.kaggle.com/yasufuminakama) who shared great starter notebooks. This notebook uses the preprocessing notebook by Y.Nakama to extract the number of atoms of each element

In [ ]:
import os
import gc
import re
import numpy as np
import pandas as pd
from collections import OrderedDict
from tqdm.autonotebook import tqdm

In [ ]:
train = pd.read_pickle('../input/inchi-preprocess-2/train2.pkl')

def get_train_file_path(image_id):
    return "../input/bms-molecular-translation/train/{}/{}/{}/{}.png".format(
        image_id[0], image_id[1], image_id[2], image_id 
    )

train['file_path'] = train['image_id'].apply(get_train_file_path)

print(f'train.shape: {train.shape}')
train.head()

In [ ]:
import cv2
import matplotlib.pyplot as plt
plt.imshow(cv2.imread(train.loc[2, 'file_path'])[..., ::-1])

In [ ]:
train['formula'] = train['InChI_text'].str.split("/", expand=True, n=1).drop(1, axis=1).rename(columns={0: "formula"})

In [ ]:
train = train[['formula', 'file_path', 'InChI_length']]

In [ ]:
atoms = ['B', 'Br', 'C', 'Cl', 'F', 'H', 'I', 'N', 'O', 'P', 'S', 'Si']
for atom in atoms:
    train[atom] = -1

train.head()

In [ ]:
def get_counts(row):    
    counts = OrderedDict((atom, 0) for atom in atoms)
    formula = row.formula.strip().split()
    for i, item in enumerate(formula):
        if item not in atoms: continue
        elif (i + 1) == len(formula):
            counts[item] = 1
        elif formula[i + 1] in atoms:
            counts[item] = 1
        else:
            counts[item] = int(formula[i + 1])
    return list(counts.values())

In [ ]:
tqdm.pandas()
out = train.progress_apply(get_counts, axis=1)

In [ ]:
train.iloc[:, 3:] = np.stack(out.values)

In [ ]:
train.sample(5)

In [ ]:
train.to_pickle("cnn_pretrain.pkl")

In [ ]:
pd.read_pickle("cnn_pretrain.pkl")